In [1]:
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import KFold
import pickle

In [2]:
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')

In [3]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)
climates = dfc['climate'].unique()            ### split by climate
C = []
for clima in climates:
    V=[]
    tmp = dfc[dfc['climate']==clima].reset_index(drop=True)
    tmp = tmp.sort_values('PopDens', ascending=False).reset_index(drop=True)
    for train_index, val_index in kf.split(tmp.PopDens):
        V.append(tmp[tmp.index.isin(val_index)])
    C.append(V)

In [4]:
############ Test phase ###################
df = pd.DataFrame()
cities_ = []
for j in range(0,5):
    cities = []
    for i in range(0, 4):
        cities.extend(list(C[i][j]['UC_NM_MN']))
    cities_.append(cities)

In [5]:
############## Trainig validation ##############
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)
cities_tv = []
for phase, c in enumerate(cities_):
    V = []
    gdfF = dfc[~dfc['UC_NM_MN'].isin(c)].reset_index(drop=True)
    tmp = gdfF.sort_values('PopDens', ascending=False).reset_index(drop=True)
    for train_index, val_index in kf.split(tmp.PopDens):
        V.append(tmp[tmp.index.isin(val_index)])
    
    for v in range(0,len(V)):
        cities = list(V[v]['UC_NM_MN'])
        cities_tv.append(cities)

In [6]:
with open('../data_revision/cities/cv/test.pkl', 'wb') as f:
    pickle.dump(cities_, f)

In [7]:
with open('../data_revision/cities/cv/train_val.pkl', 'wb') as f:
    pickle.dump(cities_tv, f)